In [7]:
import setup_notebook

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error

from grandexchange.preprocess import load_preprocessed_data

# How many steps in the future to predict
N_PREDICT_STEPS = 7

# Number of steps for the first cross validation fold
MIN_CV_SIZE = 500

### Load data

In [60]:
data = load_preprocessed_data(6)

# For now take a sample of 5 items to test model
data = data[
    data["item_id"].isin(
        data["item_id"].sample(5, random_state=42)
    )
].reset_index(drop=True)

# Make wide
data = data[["name", "datetime", "price"]].pivot(index="datetime", columns="name", values="price").reset_index()
data.columns.name = None

Local DB found, checking for updated grandexchange...
No new data found


In [61]:
data

,datetime,Blue d'hide body,Dragon bolts,Fire battlestaff,Mahogany logs,Sanfew serum(4)
0,2022-01-01 00:00:00,4873.083333,3669.083333,8922.916667,360.500000,31534.083333
1,2022-01-01 06:00:00,4945.583333,3673.583333,8910.500000,360.000000,31721.000000
2,2022-01-01 12:00:00,5046.666667,3658.916667,8980.416667,355.250000,32282.166667
3,2022-01-01 18:00:00,4956.500000,3621.916667,8967.333333,344.916667,32501.916667
4,2022-01-02 00:00:00,4813.000000,3573.166667,8916.166667,341.333333,33626.500000
...,...,...,...,...,...,...
1054,2022-09-21 12:00:00,5059.416667,1791.583333,8951.416667,232.166667,39800.082561
1055,2022-09-21 18:00:00,4937.166667,1791.583333,8909.416667,220.500000,41099.833333
1056,2022-09-22 00:00:00,4989.583333,1782.916667,8944.166667,216.916667,42366.416667
1057,2022-09-22 06:00:00,5113.000000,1770.083333,8925.250000,223.250000,43784.250000


### Cross-validation

In [64]:
from sktime.forecasting.model_selection import SlidingWindowSplitter

df = data
FORECAST_LENGTH = 7
STEP_LENGTH = 150
N_STEPS = data["datetime"].nunique()

def get_cv_splits(n_steps: int, forecast_length: int, step_length: int):
    window_size = n_steps // 2
    splitter = SlidingWindowSplitter(
        fh = list(range(1, forecast_length + 1)),
        window_length=window_size,
        step_length=step_length
    )
    return list(splitter.split(np.arange(n_steps)))

In [65]:
cv_split = 0

cv_splits = get_cv_splits(N_STEPS, FORECAST_LENGTH, STEP_LENGTH)

train = data.iloc[cv_splits[cv_split][0]]
test = data.iloc[cv_splits[cv_split][1]]

### Model